<a href="https://colab.research.google.com/github/teamsimcoemmai/mmai894/blob/894-pj/Final%20Project/VGG_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np

import cv2 as cv

import os
import random

import tensorflow as tf

from shutil import copyfile

from tqdm import tqdm

from fastai.vision import *
from fastai.metrics import error_rate
from fastai import *
import cv2 as cv
import numpy as np
import pandas as pd
import scipy.io as sio

from sklearn.metrics import precision_recall_fscore_support

import shutil

In [3]:
def random_seed(seed_value, use_cuda):  #gleaned from multiple forum posts
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: torch.cuda.manual_seed_all(seed_value) # gpu 


random_seed(42,True)

data = ImageDataBunch.from_folder('/content/gdrive/MyDrive/Simcoe/894/Car dataset/new','train_set','val_set','test_set',
                                  seed=42,
                                  ds_tfms=get_transforms(do_flip=True, 
                                                         flip_vert=True, 
                                                         p_affine=0.75, # Default value
                                                         max_rotate=5.0, 
                                                         max_zoom=1.1,
                                                         p_lighting=0.75, # Defalut value 
                                                         max_lighting=0.2
                                                         ),
                                  size=224,
                                  bs=32).normalize(imagenet_stats)

In [4]:
df_test = pd.read_csv('/content/gdrive/MyDrive/Simcoe/894/Car dataset/df_test_new.csv')

In [5]:
def test_predict(df_test):

  '''
  Args
  df_test: Test data df

  Return:
  Data frame with predicted and actual for each image from the test data set   
  '''

#Predict from test data set
  prediction,y = learn.get_preds(DatasetType.Test)

#Map predictions to image name on test data set
  preds = []
  dataIndex = []
  num_preds = len(prediction)
  for i in range(num_preds):
    p = learn.predict(data.test_ds.x[i])
    preds.append(str(p[0]))
    dataIndex.append(str(learn.data.test_ds.items[i]).split('/')[-1])

#Convert predictions from string to Int 
  preds = list(map(int, preds))

#Zip image name and predictions
  pred_tup = list(zip(dataIndex,preds))

#Create data frame from image name and predictions
  df_pred = pd.DataFrame(pred_tup,columns=['img_name','pred_class'])

#Load test data set - Data has already been pre processed and uploaded into drive
  actual_tup = df_test[['img_name','class']].to_records(index=False)

# Join preictions test daframe and predictions data fram
  df_valid = df_test.merge(df_pred, left_on='img_name', right_on='img_name')

  feature_list = ['img_name','class','pred_class']
  df_valid = df_valid[feature_list]

  return df_valid




In [6]:
def precision_recall_fscore(df):

  '''
  Function to calculate precision, recall, fscore
  '''

  precision, recall, fscore, support = precision_recall_fscore_support(df['class'].values,
                                                                     df['pred_class'].values, 
                                                                     average = 'weighted')
  
  return precision, recall, fscore


In [7]:
def calc_accuracy(df):

  '''
  Function to calculate accuracy
  '''

  # Find accuracy from test data set
  df['diff'] = np.where(df['class'] == df['pred_class'],1,0)
  accuracy = df['diff'].sum()/df.shape[0]*100

  return accuracy

In [8]:
learn = create_cnn(data, 
                   models.vgg16_bn,
                   metrics=[accuracy])

/usr/local/lib/python3.7/dist-packages/fastai/vision/learner.py:109: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


In [9]:
from fastai.callbacks import EarlyStoppingCallback

# Callback to avoid over fitting
callback = EarlyStoppingCallback(learn,
                                  monitor = 'accuracy',
                                  patience = 2)                                               
                                

In [10]:
from fastai.callbacks import *

# Transfer learning from pretrained models
#Hyper paramer tuning - fit_one_cycle method varies learing rate to get the best accuracy
learn.fit_one_cycle(10,
                    callbacks = callback
                    )

epoch,train_loss,valid_loss,accuracy,time
0,5.910659,4.570137,0.091845,45:51
1,4.316960,3.300694,0.248764,02:09
2,3.421831,2.632601,0.367792,02:07
3,2.917916,2.189312,0.437397,02:07
4,2.509912,1.907520,0.512356,02:04
5,2.253070,1.632270,0.586079,02:05
6,1.934736,1.542023,0.592669,02:03
7,1.785828,1.425825,0.626853,02:03
8,1.676791,1.387172,0.637150,02:03
9,1.608713,1.380801,0.635914,02:02


In [11]:
learn.save('/content/gdrive/MyDrive/Simcoe/894/saved_models/vgg16_fastai')

In [12]:
learn.load('/content/gdrive/MyDrive/Simcoe/894/saved_models/vgg16_fastai')

Learner(data=ImageDataBunch;

Train: LabelList (11330 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
152,152,152,152,152
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new;

Valid: LabelList (2428 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
192,192,192,192,192
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new;

Test: LabelList (2427 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: EmptyLabelList
,,,,
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv

In [13]:
df_test_predict = test_predict(df_test)

   Score after Transfer learing
   

In [14]:
precision, recall, fscore = precision_recall_fscore(df_test_predict)

print("Precision : " + str(precision))
print("Recall: " + str(recall))
print("F1 Score : " + str(fscore))

Precision : 0.6762541399420876
Recall: 0.6468891635764318
F1 Score : 0.6467791559334924


In [15]:
accuracy = calc_accuracy(df_test_predict)

print("Accuracy :" + str(accuracy))

Accuracy :64.68891635764318


In [22]:
#Unfreeze model

learn.unfreeze()

#Hyper paramer tuning - fit_one_cycle method varies learing rate to get the best accuracy
learn.fit_one_cycle(100, max_lr = slice(2e-5, 2e-4), wd = 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.498868,0.591959,0.838550,02:10
1,0.480904,0.583095,0.841845,02:12
2,0.470770,0.574084,0.841433,02:13
3,0.459137,0.574705,0.842669,02:10
4,0.471537,0.563931,0.840610,02:11
5,0.434566,0.560466,0.843493,02:11
6,0.442957,0.550677,0.843081,02:11
7,0.443060,0.547955,0.849671,02:13
8,0.403609,0.536659,0.848847,02:13
9,0.419788,0.532305,0.850082,02:13


In [23]:
learn.save('/content/gdrive/MyDrive/Simcoe/894/saved_models/vgg16_unfreeze')

In [24]:
learn.load('/content/gdrive/MyDrive/Simcoe/894/saved_models/vgg16_unfreeze')

Learner(data=ImageDataBunch;

Train: LabelList (11330 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
152,152,152,152,152
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new;

Valid: LabelList (2428 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
192,192,192,192,192
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new;

Test: LabelList (2427 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: EmptyLabelList
,,,,
Path: /content/gdrive/MyDrive/Simcoe/894/Car dataset/new, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv

In [25]:
df_test_predict = test_predict(df_test)

Scores after unfreezing

In [26]:
precision, recall, fscore = precision_recall_fscore(df_test_predict)

print("Precision : " + str(precision))
print("Recall: " + str(recall))
print("F1 Score : " + str(fscore))

Precision : 0.9251344331690042
Recall: 0.9180057684384013
F1 Score : 0.918063188197208


In [27]:
accuracy = calc_accuracy(df_test_predict)

print("Accuracy :" + str(accuracy))

Accuracy :91.80057684384013
